In [46]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-132652")
exp = Experiment(workspace=ws, name="udacity-project-hyper")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-132652
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-132652


In [47]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster-2"
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing Comput Target")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

### YOUR CODE HERE ###

Found existing Comput Target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [48]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, loguniform
import os

# Specify parameter sampler
ps1 = RandomParameterSampling(
     {"--C": loguniform(-3,2), "--max_iter": choice(1,5,10,20,30,40,50,80,100,200,400,800,1000)}
)

### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
arguments = {"--C": 1.0, "--max_iter": 100}
current_dir = os.getcwd()
est = SKLearn(source_directory=current_dir, 
              entry_script="train.py",
              compute_target = compute_target,
              script_params = arguments)

### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps1,
    primary_metric_name="Accuracy",
    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=2
)

In [49]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run_hyper = exp.submit(config=hyperdrive_config)
### YOUR CODE HERE ###

In [ ]:
run_hyper.wait_for_completion(show_output=True)

RunId: HD_f5269347-240f-45d5-8554-dc1c5b9aeb9e
Web View: https://ml.azure.com/experiments/udacity-project-hyper/runs/HD_f5269347-240f-45d5-8554-dc1c5b9aeb9e?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-132652/workspaces/quick-starts-ws-132652

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-31T17:23:54.124685][API][INFO]Experiment created<END>\n""<START>[2020-12-31T17:23:54.748461][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space<END>\n""<START>[2020-12-31T17:23:54.890359][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-31T17:23:55.6981247Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [ ]:
import joblib
best_run = run_hyper.get_best_run_by_primary_metric()

In [ ]:
print(best_run.get_file_names())

In [ ]:
model = best_run.register_model(workspace=ws, model_name="best-hyper-model", model_path="outputs/model.joblib")

In [26]:
dir(best_run)

['DELIM',
 'EXPERIMENT_PATH',
 'RUN_PATH',
 'WORKSPACE_FMT',
 '_DEFAULT_GET_CONTENT_TIMEOUT',
 '_RUNSOURCE_PROPERTY',
 '_WAIT_COMPLETION_POLLING_INTERVAL_MAX',
 '_WAIT_COMPLETION_POLLING_INTERVAL_MIN',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cleanup',
 '_client',
 '_container',
 '_context_manager',
 '_create',
 '_download_artifact_contents_to_string',
 '_dto_to_run',
 '_experiment',
 '_experiment_url',
 '_get_base_info_dict',
 '_get_blob_azureml_datastore',
 '_get_last_log_primary_instance',
 '_get_logs',
 '_get_outputs_datapath',
 '_get_portal_domain',
 '_heartbeat',
 '_identity',
 '_kill',
 '_load_scope',
 '_log_context',
 '_logge

In [22]:
import joblib
# Get your best run and save the model from that run.
best_run = run_hyper.get_best_run_by_metric()
model = best_run.register_model(model_name="best-hyper-model", "outputs/model")

SyntaxError: positional argument follows keyword argument (<ipython-input-22-d5c24c770e18>, line 4)

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv", header=True, validate=True)

In [6]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x['y'] = y

In [10]:
from azureml.core import Dataset
dataset_in = Dataset.from_pandas_dataframe(x)

In [17]:
dataset_in = dataset_in.register(workspace=ws, name="auto-ml-training-data", description="example exercise data")

In [18]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target = compute_target,
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=dataset_in,
    label_column_name="y",
    n_cross_validations=5)

In [19]:
# Submit your automl run
run_auto = exp.submit(automl_config, show_output=False)

Running on remote.


In [20]:
run_auto.wait_for_completion(show_output=True)



ERROR: Setup iteration failed: Unidentified error, check logs in portal / compute


{'runId': 'AutoML_379473b2-1c84-44c0-8562-638f56458135',
 'target': 'cpu-cluster-2',
 'status': 'Failed',
 'startTimeUtc': '2020-12-31T16:54:31.589637Z',
 'endTimeUtc': '2020-12-31T16:57:46.421647Z',
 'error': {'error': {'code': 'System',
   'message': '\nError Code: StepTranslation\nValidation Error Code: StepNotReady\nValidation Target: Step(Microsoft.DPrep.ReferenceBlock)\nFailed Step: b0046e2b-f21d-4f6a-bd34-87ac5cb0ca37\nFailed Step Type: Microsoft.DPrep.ReferenceBlock\nError Message: Referenced activity failed to resolve\nFailed to resolve reference. FailureReason: ActivityMissing\n| session_id=bb8c40f9-4ab5-44b4-8b85-9f5990e92d6c',
   'messageFormat': 'Failed to get data from DataPrep. ErrorCode: StepTranslation. Message: Referenced activity failed to resolve\nFailed due to inner exception of type: ActivityReferenceResolutionException\n| session_id=bb8c40f9-4ab5-44b4-8b85-9f5990e92d6c',
   'messageParameters': {},
   'detailsUri': 'https://aka.ms/azureml-known-errors',
   'detai

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###